In [1]:
import os

import numpy as np
import tensorflow as tf
import math
from models.model_type import MODELS
from utils.data_utils import DatasetVectorizer
from utils.other_utils import init_config
from utils.other_utils import logger


dirs = ['./Annotation/Saurabh','./Annotation/Rachna + K','./Annotation/Kaveri','./Annotation/Zubair','./Annotation/Saujas']


QUESTIONS = ["qy", "qw", "qo", "qr","qh"]

def get_similarity(sentence1, sentence2):
    return(cosine(model.encode(sentence1)[0], model.encode(sentence2)[0]))
    
    # similarity_processor.get_similarity(sentence1, sentence2)


def is_question(utterance):
    try:
        if utterance.split("|")[2].strip(" ") in QUESTIONS: 
            return 1
    except:
        return 0

/home/saurabh/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Colocations handled automatically by placer.


In [2]:
def manhattan_similarity(x1,x2):
        
    manhattan_sim = np.exp(-manhattan_distance(x1, x2))

    return manhattan_sim

def manhattan_distance(x,y):
 
    return sum(abs(a-b) for a,b in zip(x,y))

In [3]:
class MultiheadSiameseNetGuiDemo:

    def __init__(self):

        self.resultLabel = {'Result' : 0}


        self.main_config = init_config()
        self.model_dir = str(self.main_config['DATA']['model_dir'])

        model_dirs = [os.path.basename(x[0]) for x in os.walk(self.model_dir)]


        self.vectorizer = DatasetVectorizer(self.model_dir)

        self.max_doc_len = self.vectorizer.max_sentence_len
        self.vocabulary_size = self.vectorizer.vocabulary_size

        self.session = tf.Session()
        self.model = self.load_model1("rnn_64")

    def predict(self,sentences1,sentences2):
        if self.model:
            sentence1 = sentences1
            sentence2 = sentences2
            x1_sen = self.vectorizer.vectorize(sentence1)
            x2_sen = self.vectorizer.vectorize(sentence2)
            feed_dict = {self.model.x1: x1_sen, self.model.x2: x2_sen,
                         self.model.is_training: False}
            prediction = np.squeeze(self.session.run([self.model.predictions], feed_dict=feed_dict))
            prediction = np.round(prediction, 2)
            self.resultLabel['text'] = prediction
            
            return prediction


 


    def load_model(self, model_name):
        logger.info('Loading model: %s', model_name)

        model = MODELS[model_name.split('_')[0]]
        model_config = init_config(model_name.split('_')[0])

        self.model = model(self.max_doc_len, self.vocabulary_size, self.main_config, model_config)
        saver = tf.train.Saver()
        last_checkpoint = tf.train.latest_checkpoint('{}/{}'.format(self.model_dir, model_name))
        saver.restore(self.session, last_checkpoint)
        logger.info('Loaded model from: %s', last_checkpoint)
        saver.save(self.session, 'my_test_model',global_step=1000)
        pos = 0
        count = 0
        for x in dirs: 
            for file in os.listdir(x):
                f = open(x+'/'+file,'r')
                print(f.name)
                lines =f.readlines()
                scan_range = 10
                for index, utterance in enumerate(lines):
                    if index < scan_range or index >= len(lines) - scan_range:
                        continue
                    if is_question(utterance):
                        a1 = []
                        a2 = []
                        for i in range(-scan_range,0):
                            if len((lines[index+i]).split("|")[1]) > 0:
                                a1.append(self.predict(lines[index].split("|")[1], lines[index+i].split("|")[1]))
                        for i in range(1,scan_range+1):
                            if len((lines[index+i]).split("|")[1]) > 0:
                                a2.append(self.predict(lines[index].split("|")[1], lines[index+i].split("|")[1]))
#                         a1 = [x for x in a1 if x > 0.68]
#                         a2 = [x for x in a2 if x > 0.68]
                        keys = ['sbd','sb-d','sd','sw-d']
                        if sum(a1) - sum(a2) > 0.3  :
                            count += 1
                            if 's' in lines[index].split("|")[3].strip(" "):
                                pos += 1
                            else:
                                print("Predicted wrong Sentence : " + lines[index] + " Predicted Label : sbd")

                        if sum(a2) - sum(a1) > 0.2 :
                            count += 1
                            if 'i' in lines[index].split("|")[3].strip(" ").lower():
                                pos += 1
                            else:
                                print("Predicted wrong Sentence : " + lines[index] + " Predicted Label : i")
        print(pos/count)   
        
    def load_model1(self, model_name):
            logger.info('Loading model: %s', model_name)

            model = MODELS[model_name.split('_')[0]]
            model_config = init_config(model_name.split('_')[0])

            self.model = model(self.max_doc_len, self.vocabulary_size, self.main_config, model_config)
            saver = tf.train.Saver()
            last_checkpoint = tf.train.latest_checkpoint('{}/{}'.format(self.model_dir, model_name))
            saver.restore(self.session, last_checkpoint)
            logger.info('Loaded model from: %s', last_checkpoint)
            saver.save(self.session, 'my_test_model',global_step=1000)
            f = open('./betterrachna.txt','r')
            lines =f.readlines()
            scan_range = 5
            topics = []
            newLines = []
            for index, utterance in enumerate(lines):
                
                if index < scan_range or index >= len(lines) - scan_range:
                    continue
                
                j = index
                k = index + 3
                previous = []
                forward = []
                while j < index + 3:
                    a1 = []
                    a2 = []

                    for i in range(-scan_range,0):
                            a1.append(self.predict(lines[j].split("|")[1], lines[index+i].split("|")[1]))


                    for i in range(1,scan_range+1):
                            a2.append(self.predict(lines[j].split("|")[1], lines[k+i].split("|")[1]))

                    
                    previous.append(a1)
                    forward.append(a2)
                    j += 1           
                previousAvg = np.mean(previous,axis = 0)
                forwardAvg = np.mean(forward,axis = 0)
                print("Previous Average of Sentence 1: ",previous[0])
                print("The three sentences are : ",lines[index],lines[index+1],lines[index+2])
                print("Previous Average : ",previousAvg)
                print("Forward Average : ",forwardAvg)
                print("Manhattan Similarity : ", manhattan_similarity(previousAvg,forwardAvg))
                    


In [4]:
gui = MultiheadSiameseNetGuiDemo()


INFO:tensorflow:Reading main configuration.
INFO:tensorflow:Loading model: rnn_64
INFO:tensorflow:Reading configuration for rnn model.
Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from model_dir/rnn_64/model-6380
INFO:tensorflow:Loaded model from: model_dir/rnn_64/model-6380
Instructions for updating:
Please use tensorflow/transform or tf.data.
Previous Average of Sentence 1:  [0.09, 0.12, 0.33, 0.06, 0.1]
The three sentences are :  B|And I don't know too much about the air pollution thing.|sd |1-
 B|I do know for other

Previous Average of Sentence 1:  [0.0, 0.01, 0.0, 0.0, 0.0]
The three sentences are :  A|And so the Sherman deal is, um, uh, lost a lot of hair over that project.|sd
 A|Uh, what we run into, um, is we have the Texas Air Control Board, T A C B, that send out, uh, jurisdictions under which we have to, uh, uh, reply to.|sd
 A|And a lot of their rules and regulation aren't real clear, so we have our manager of environmental, who assist the T A C B, which is located in Austin, in writing|sd

Previous Average :  [0.3033333  0.39666665 0.35666665 0.35999998 0.40666667]
Forward Average :  [0.04       0.39333335 0.07       0.33       0.4       ]
Manhattan Similarity :  0.5543273323128015
Previous Average of Sentence 1:  [0.59, 0.5, 0.59, 0.65, 0.0]
The three sentences are :  A|Uh, what we run into, um, is we have the Texas Air Control Board, T A C B, that send out, uh, jurisdictions under which we have to, uh, uh, reply to.|sd
 A|And a lot of their rules and regulation aren't real clear, so we 

Previous Average of Sentence 1:  [0.46, 0.44, 0.66, 0.29, 0.54]
The three sentences are :  B|Now is that place built|qy
 B|or you're still on the makings of it?|qrr
 A|The Sherman facility, we are still in the, closer too, but still have a little bit of, um, finalization to do.|sd

Previous Average :  [0.26666668 0.26333335 0.37       0.19333334 0.32666668]
Forward Average :  [0.25666666 0.02333333 0.00666667 0.25333333 0.28333333]
Manhattan Similarity :  0.4883774478636945
Previous Average of Sentence 1:  [0.27, 0.38, 0.2, 0.34, 0.44]
The three sentences are :  B|or you're still on the makings of it?|qrr
 A|The Sherman facility, we are still in the, closer too, but still have a little bit of, um, finalization to do.|sd
 A|It's been pitched to the Sherman City Council with, um, open arms, as well as the public.|sd

Previous Average :  [0.12       0.15333332 0.1        0.14999999 0.17333333]
Forward Average :  [0.02333333 0.00666667 0.12       0.12666667 0.16333333]
Manhattan Similarity

Previous Average of Sentence 1:  [0.83, 0.02, 0.58, 0.07, 0.14]
The three sentences are :  A|and, uh, it's an interesting job.|sd |3-
 B|Well, I know from some of the sites that we've had, uh, quite a list of cites that have gone bad|sd |4+
 B|and you have to clean up.|sd

Previous Average :  [0.6166666  0.02666667 0.42333332 0.07666667 0.18666667]
Forward Average :  [0.         0.58666664 0.53999996 0.40666667 0.42666665]
Manhattan Similarity :  0.15515460225749636
Previous Average of Sentence 1:  [0.03, 0.4, 0.08, 0.19, 0.65]
The three sentences are :  B|Well, I know from some of the sites that we've had, uh, quite a list of cites that have gone bad|sd |4+
 B|and you have to clean up.|sd
 B|And, you know the law now is the super fund|sd

Previous Average :  [0.02333333 0.37666667 0.06666666 0.15666668 0.47666666]
Forward Average :  [0.46000004 0.4433333  0.33666667 0.35000002 0.01      ]
Manhattan Similarity :  0.23851254935242472
Previous Average of Sentence 1:  [0.29, 0.08, 0.23, 0

Previous Average of Sentence 1:  [0.8, 0.1, 0.79, 0.77, 0.57]
The three sentences are :  A|we have duckwork and all those type of interesting pieces of equipment where, um, they have, um, that we sample|sd
 B|Uh-huh.|sd
 A|and it may have over, uh, a certain emission levels,|sd

Previous Average :  [0.84      0.1       0.73      0.7266666 0.58     ]
Forward Average :  [0.84       0.28       0.47666666 0.6233333  0.8       ]
Manhattan Similarity :  0.4692278771670236
Previous Average of Sentence 1:  [0.1, 0.77, 0.77, 0.58, 0.9]
The three sentences are :  B|Uh-huh.|sd
 A|and it may have over, uh, a certain emission levels,|sd
 A|this is on the solid side ,|sd

Previous Average :  [0.1        0.50333333 0.51       0.44999996 0.59      ]
Forward Average :  [0.3        0.35999998 0.45000002 0.57       0.29666665]
Manhattan Similarity :  0.441902198170775
Previous Average of Sentence 1:  [0.63, 0.64, 0.59, 0.75, 0.78]
The three sentences are :  A|and it may have over, uh, a certain emission 

Previous Average of Sentence 1:  [0.74, 0.83, 0.87, 0.45, 0.66]
The three sentences are :  A|Yeah.|ny
 B|It's a heck of a lot different.|sd
 B|And we used to be really embarrassed about the gray metal desk.|sd

Previous Average :  [0.68       0.6466667  0.6166667  0.50333333 0.58666664]
Forward Average :  [0.07333333 0.24       0.53       0.         0.39666665]
Manhattan Similarity :  0.16640454933473567
Previous Average of Sentence 1:  [0.44, 0.36, 0.55, 0.46, 0.41]
The three sentences are :  B|It's a heck of a lot different.|sd
 B|And we used to be really embarrassed about the gray metal desk.|sd
 B|We were about the only, place in, you know, T I that had the gray metal desk.|sd

Previous Average :  [0.44333336 0.39333335 0.46333334 0.46       0.42666665]
Forward Average :  [0.23       0.42333332 0.00333333 0.41333333 0.43666664]
Manhattan Similarity :  0.46766642450115103
Previous Average of Sentence 1:  [0.62, 0.51, 0.64, 0.65, 0.48]
The three sentences are :  B|And we used to be r

IndexError: list index out of range

In [ ]:
# file = open('./rachna.txt','r')
# nope = ['b','%','aa','bk','fo_o_fw_"_by_bc']
# yo = []
# for line in file.readlines():
#     if line.split("|")[2].strip("\n").strip(" ") not in nope:
#         yo.append(line)

In [ ]:
# with open('./betterrachna.txt', 'w') as f:
#     for s in yo:
#         f.write(s)



In [ ]:
# yo